In [227]:
import requests
import datetime
import base64
from urllib.parse import urlencode
import pandas as pd
import numpy as np
import time


In [2]:
client_id = '86241dc85057485b9d020a85673e5c1d'
client_secret = '67e6e25d4d5c43e3abb36860e8fc1355'

In [4]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_did_expire = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        if client_secret == None or client_id == None:
            raise Exception("You must set client_id and client_secret")
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        } 
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Could not authenticate client.")
            # return False
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_did_expire = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
        
        
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, _id):
        return self.get_resource(_id, resource_type='albums')
    
    def get_artist(self, _id):
        return self.get_resource(_id, resource_type='artists')
    
    def base_search(self, query_params): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        lookup_url = f"{endpoint}?{query_params}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json()
    
    def search(self, query=None, operator=None, operator_query=None, search_type='artist' ):
        if query == None:
            raise Exception("A query is required")
        if isinstance(query, dict):
            query = " ".join([f"{k}:{v}" for k,v in query.items()])
        if operator != None and operator_query != None:
            if operator.lower() == "or" or operator.lower() == "not":
                operator = operator.upper()
                if isinstance(operator_query, str):
                    query = f"{query} {operator} {operator_query}"
        query_params = urlencode({"q": query, "type": search_type.lower()})
        print(query_params)
        return self.base_search(query_params)

In [5]:
spotify = SpotifyAPI(client_id, client_secret)

In [348]:
spotify.perform_auth()

True

In [8]:
spotify.access_token

'BQB16-PK6-_Sf-uoEVqlQhVlCju2A-AydKmvOz0ttbWAyBFLZNzv0Ygi3EaFJNCQTSEkSJmHcjedTQ__fDo'

In [290]:
stronger = spotify.search({"track": "Bad Habits" , "artist":"Ed Sheeran"}, search_type='Track')

q=track%3ABad+Habits+artist%3AEd+Sheeran&type=track


In [291]:
stronger

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3ABad+Habits+artist%3AEd+Sheeran&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6eUKZXaKkcviH0Ku9w2n3V'},
       'href': 'https://api.spotify.com/v1/artists/6eUKZXaKkcviH0Ku9w2n3V',
       'id': '6eUKZXaKkcviH0Ku9w2n3V',
       'name': 'Ed Sheeran',
       'type': 'artist',
       'uri': 'spotify:artist:6eUKZXaKkcviH0Ku9w2n3V'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
  

In [ ]:
type(stronger)

In [304]:
list(stronger)

['tracks']

In [305]:
type(stronger['tracks'])

dict

In [306]:
list(stronger['tracks'])

['href', 'items', 'limit', 'next', 'offset', 'previous', 'total']

In [312]:
type(stronger['tracks']["items"][0]["album"]['release_date'])

str

In [313]:
stronger['tracks']["items"][0]["album"]['release_date']

'2021-10-29'

In [ ]:
list(stronger['tracks']['items'])

In [ ]:
type(stronger['tracks']['items'][0])

In [ ]:
list(stronger['tracks']['items'][0])

In [ ]:
stronger['tracks']['items'][0]["name"]

In [187]:
tracks = pd.read_csv('data/hot_100.csv')

In [166]:
tracks.head()

,date,rank,song,artist,last-week,peak-rank,weeks-on-board
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18


In [176]:
songandartist= tracks[["song","artist"]]

In [180]:
tracks[["song","artist"]].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [184]:
yy = np.unique(tracks[["song","artist"]].values)

AttributeError: 'numpy.ndarray' object has no attribute 'to_dataframe'

In [179]:
songandartist.nunique()

song      24620
artist    10205
dtype: int64

In [ ]:
tracks["song"][0]

In [ ]:
tracks["artist"][0]

In [ ]:
sp_song = tracks["song"][0]
sp_artist = tracks["artist"][0]

In [ ]:
spotify.search({"track": "Bad Habits" , "artist":"Ed Sheeran"}, search_type='Track')

In [ ]:
spot_song = spotify.search({"track": sp_song , "artist": sp_artist}, search_type='Track')

In [ ]:
spot_song['tracks']['items'][0]["name"]

In [42]:
def get_songg(x , y):
    sp_song = x
    sp_artist = y
    #spot_song = spotify.search({"track": sp_song , "artist": sp_artist}, search_type='Track')
    return {"track": sp_song , "artist": sp_artist}

In [41]:
spot_song

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AStay+artist%3AThe+Kid+LAROI+%26+Justin+Bieber&type=track&offset=0&limit=20',
  'items': [],
  'limit': 20,
  'next': None,
  'offset': 0,
  'previous': None,
  'total': 0}}

In [59]:
len(tracks)

330087

In [93]:
tracks.iloc[0]

date              2021-11-06
rank                       1
song              Easy On Me
artist                 Adele
last-week                  1
peak-rank                  1
weeks-on-board             3
spot_song         Easy On Me
test                       0
8                          0
Name: 0, dtype: object

In [164]:
tracks[["song","artist"]].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [277]:
tracks["test1"] = None
tracks["artist_test"] = None



In [293]:
tracks["duration_ms"] = None

In [314]:
tracks["release_date"] = None

In [319]:
tracks["song_id"] = None
tracks["Album_id"] = None

In [332]:
tracks["artist_id"] = None

In [ ]:
x = 1

In [285]:
sleeptimer = 10

In [339]:
def get_info(i):
    sp_song = tracks.loc[i,"song"]
    sp_artist = tracks.loc[i,"artist"]
    spot_song = spotify.search({"track": sp_song }, search_type='Track')
    tracks.loc[i,"test1"] = spot_song['tracks']['items'][0]["name"]
    tracks.loc[i,"artist_test"] =spot_song['tracks']['items'][0]['artists'][0]['name']
    tracks.loc[i,"duration_ms"] = spot_song['tracks']['items'][0]["duration_ms"]
    tracks.loc[i,"release_date"] = spot_song['tracks']["items"][0]["album"]['release_date']
    tracks.loc[i,"Album_id"] = spot_song['tracks']["items"][0]["album"]['id']
    tracks.loc[i,"song_id"] = spot_song['tracks']['items'][0]["id"]
    tracks.loc[i,"artist_id"] = spot_song['tracks']["items"][0]["artists"][0]['id']

    

In [330]:
    spot_song = spotify.search({"track": "Industry Baby" }, search_type='Track')


q=track%3AIndustry+Baby&type=track


In [331]:
spot_song

{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3AIndustry+Baby&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/7jVv8c5Fj3E9VhNjxT4snq'},
       'href': 'https://api.spotify.com/v1/artists/7jVv8c5Fj3E9VhNjxT4snq',
       'id': '7jVv8c5Fj3E9VhNjxT4snq',
       'name': 'Lil Nas X',
       'type': 'artist',
       'uri': 'spotify:artist:7jVv8c5Fj3E9VhNjxT4snq'},
      {'external_urls': {'spotify': 'https://open.spotify.com/artist/2LIk90788K0zvyj2JJVwkJ'},
       'href': 'https://api.spotify.com/v1/artists/2LIk90788K0zvyj2JJVwkJ',
       'id': '2LIk90788K0zvyj2JJVwkJ',
       'name': 'Jack Harlow',
       'type': 'artist',
       'uri': 'spotify:artist:2LIk90788K0zvyj2JJVwkJ'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE'

In [340]:
get_info(2)

q=track%3AIndustry+Baby&type=track


In [160]:
tracks.loc[150000,"test1"]

In [44]:
tracks["spot_song"] = tracks["spot_song"].apply(tracks["song"], tracks["artist"])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [341]:
tracks.head(10)

,date,rank,song,artist,last-week,peak-rank,weeks-on-board,test1,artist_test,durattion,duration_ms,release_date,song_id,Album_id,artist_id
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,Easy On Me,Adele,None,224694,None,None,None,None
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,STAY (with Justin Bieber),The Kid LAROI,None,None,None,None,None,None
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,INDUSTRY BABY (feat. Jack Harlow),Lil Nas X,None,212000,2021-07-23,27NovPIUIRrOZoCHxABJwK,622NFw5Yk0OReMJ2XWcXUh,7jVv8c5Fj3E9VhNjxT4snq
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,Fancy Like,Walker Hayes,None,None,None,None,None,None
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,Bad Habits,Ed Sheeran,None,None,None,None,None,None
5,2021-11-06,6,Way 2 Sexy,Drake Featuring Future & Young Thug,6.0,1,8,Way 2 Sexy (with Future & Young Thug),Drake,None,None,None,None,None,None
6,2021-11-06,7,Shivers,Ed Sheeran,9.0,7,7,None,None,None,None,None,None,None,None
7,2021-11-06,8,Good 4 U,Olivia Rodrigo,7.0,1,24,None,None,None,None,None,None,None,None
8,2021-11-06,9,Need To Know,Doja Cat,11.0,9,20,None,None,None,None,None,None,None,None
9,2021-11-06,10,Levitating,Dua Lipa,8.0,2,56,None,None,None,None,None,None,None,None


In [ ]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last-week       297775 non-null  float64
 5   peak-rank       330087 non-null  int64  
 6   weeks-on-board  330087 non-null  int64  
 7   test1           1 non-null       object 
dtypes: float64(1), int64(3), object(4)
memory usage: 20.1+ MB


In [349]:
tracks.to_csv("test.csv")

In [ ]:
x

In [ ]:
tracks['song'].count()

In [344]:
spotify.search({"track": "STAY","track":"stronger" }, search_type='Track')

q=track%3Astronger&type=track


{'tracks': {'href': 'https://api.spotify.com/v1/search?query=track%3Astronger&type=track&offset=0&limit=20',
  'items': [{'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
       'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
       'id': '5K4W6rqBFWDnAN6FQUkS6x',
       'name': 'Kanye West',
       'type': 'artist',
       'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
